<a href="https://colab.research.google.com/github/BindiyaB/DataSciencePractice/blob/master/Imputation_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [0]:
from sklearn.datasets import load_diabetes

In [0]:
diabetes=load_diabetes()

In [5]:
diabetes.DESCR

'.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - Age\n      - Sex\n      - Body mass index\n      - Average blood pressure\n      - S1\n      - S2\n      - S3\n      - S4\n      - S5\n      - S6\n\nNote: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).\n\nSource URL:\nhttps://www4.stat.ncsu.edu/~boos/var.select/diabetes.h

In [0]:
X=diabetes.data
Y=diabetes.target

In [48]:
X.shape, Y.shape

((442, 10), (442,))

In [0]:
X=X.copy() # Why do this?????????????
Xoriginal=X.copy()

In [0]:
X.ravel()[np.random.choice(X.size, 50, replace=False)] = np.nan # Randomly make 30 values as missing

In [0]:
#Function to Split data
def train_test_val_split(X,Y):
  Xtn,Xtt,Ytn,Ytt=train_test_split(X,Y,test_size=0.20)
  Xv,Xtt,Yv,Ytt=train_test_split(Xtt,Ytt,test_size=0.5)
  return Xtn,Ytn,Xtt,Ytt,Xv,Yv

In [85]:
Xtn,Ytn,Xtt,Ytt,Xv,Yv=train_test_val_split(X,Y)
Xtn.shape,Ytn.shape,Xtt.shape,Ytt.shape,Xv.shape,Yv.shape

((353, 10), (353,), (45, 10), (45,), (44, 10), (44,))

In [71]:
missing_mask = np.isnan(X)
Xoriginal[missing_mask]

array([], dtype=float64)

Imputation Technique

In [0]:
#KNN Filling
def knn_fill(Xtn,Xtt,Xv,Xoriginal):
  missing_mask_tn = np.isnan(Xtn)
  missing_mask_tt = np.isnan(Xtt)
  missing_mask_v = np.isnan(Xv)
  
  knn = KNN(k=5)
 
#   knn.fit(Xtn,Ytn)
  
#   Xtn_t=knn.transform(Xtn)
#   Xtt_t=knn.transform(Xtt)
#   Xv_t=knn.transform(Xv)
  Xtn_t=knn.fit_transform(Xtn)
  Xtt_t=knn.fit_transform(Xtt)
  Xv_t=knn.fit_transform(Xv)
  
  Xtn_mse = ((Xtn[missing_mask_tn] - Xtn_t[missing_mask_tn])**2).mean()
  Xtt_mse = ((Xtt[missing_mask_tt] - Xtt_t[missing_mask_tt])**2).mean()
  Xv_mse = ((Xv[missing_mask_v] - Xv_t[missing_mask_v])**2).mean()
  
  print("knn Impute MSE:")
  print("knn Impute MSE: %f" % Xtn_mse)
  print("knn Impute MSE: %f" % Xtt_mse)
  print("knn Impute MSE: %f" % Xv_mse)
  
  return Xtn_t,Xtt_t,Xv_t,Xtn_mse,Xtt_mse,Xv_mse

In [87]:
Xtn_t,Xtt_t,Xv_t,Xtn_mse,Xtt_mse,Xv_mse = knn_fill(Xtn,Xtt,Xv,Xoriginal)

Imputing row 1/353 with 1 missing, elapsed time: 0.032
Imputing row 101/353 with 0 missing, elapsed time: 0.033
Imputing row 201/353 with 0 missing, elapsed time: 0.033
Imputing row 301/353 with 0 missing, elapsed time: 0.034
Imputing row 1/45 with 0 missing, elapsed time: 0.001
Imputing row 1/44 with 0 missing, elapsed time: 0.001
knn Impute MSE:
knn Impute MSE: nan
knn Impute MSE: nan
knn Impute MSE: nan


In [0]:
knn_mse = ((X_filled_knn[missing_mask] - diabetes.data[missing_mask]) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)

#With k=3 --0.001336
#With k=5 -- 0.001111
#With k=10 -- 0.001134
#With k=7, -- 0.001129

#So, keep k =5
